In [26]:
from Crypto.PublicKey import RSA
from Crypto.Signature.pkcs1_15 import PKCS115_SigScheme
from Crypto.Hash import SHA256
from utils import read_file, write_to_file

# Generate 2048-bit RSA key-pair (private & public keys)
def generate_key_pair(public_key_file, private_key_file):
    keyPair = RSA.generate(2048)
    public_key = keyPair.public_key().export_key()
    private_key = keyPair.export_key()

    write_to_file(public_key_file, public_key)
    write_to_file(private_key_file, private_key)
    
    return public_key, private_key


# Load key from a file 
def load_key(key_file):
    exported_key = read_file(key_file)
    key = RSA.import_key(exported_key)
    return key


# Generate a digital signature for the given file using the private key
def sign_file(input_file, private_key):
    data = read_file(input_file)
    key = RSA.import_key(private_key)

    hash = SHA256.new(data)
    signer = PKCS115_SigScheme(key)
    
    signature = signer.sign(hash)
    return signature


# Verify the digital signature for the given message using the public key
def verify_signature(signed_file, signature, public_key):
    data = read_file(signed_file)
    key = RSA.import_key(public_key)

    hash = SHA256.new(data)
    verifier = PKCS115_SigScheme(key)

    try:
        verifier.verify(hash, signature)
        return True
    except Exception as e:
        print(e)
        return False, e


# Demo Usage
if __name__ == "__main__":

    # File paths
    input_file = "RSA Demo Files/email.txt"
    public_key_file = "RSA Demo Files/public.pem"
    private_key_file = "RSA Demo Files/private.pem"

    # Generate RSA public-private keypair
    public_key, private_key = generate_key_pair(public_key_file, private_key_file)

    # Sign file using private key
    signature = sign_file(input_file, private_key)
    print(f"File Signature: 0x{signature.hex()}")

    # Verify file signature using public key
    isValid = verify_signature(input_file, signature, public_key)
    if(isValid):
        print("Signature is valid.")
    else:
        print("Signature is invalid.")

File Signature: 0x607e6b9b8f29f306abafa9980e34b703397d2db86ab29ebb4da94d7c855a8b2e2f667ae3d825bccf366c8f3e0897debde16b983738948305cd3133f4fa44580f627abbcbc9336f41139e11f27423c5bce8025a38e57cd173f6cb106ee3d9520d7c87a13bf57d271bd8627d5064019e73957935f8a9f9012f4712b2759abb66f8ad5f225c3823b5681b679bb8a57cb2a2f674a8da498b90a9384ab717861174089222353df3446748631ae27a839e3b596a4f7c69863f81767df4a31cf09406caaa558081f01417f8c10597e760679a94804afd7b63bc5d6a56517e0cd8005947b0cf1f5f041796046f9b5b800531753c4c11f75cc8ccbf3e1797cf33ec282fd8
Signature is valid.
